# Data Wrangling: Join, Combine, and Reshape（数据规整：聚合、合并和重塑）
数据可能分散在许多文件或数据库中， 存储的形式也不利于分析。所以本章关注可以聚合、 合并、 重塑数据的方法。
### 8.1 hierarchical indexing
hierarchical indexing（层次化索引）使你能在一个轴上拥有多个（两个以上）索引级别。即：能以低维度形式处理高维度数据。

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], 
                [1, 2, 3, 1, 3, 1, 2, 2, 3]])
print(data)

a  1    1.200375
   2    0.340627
   3   -0.835507
b  1   -0.012680
   3   -0.494021
c  1   -0.991900
   2    1.200584
d  2   -0.187590
   3   -0.179506
dtype: float64


In [5]:
data.index
#codes列表里面的数字表示上面的第一个索引，0 0 0 表示：前三个索引是 a a a

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [6]:
#  部分索引
data['b']

1   -1.703054
3   -0.809895
dtype: float64

In [7]:
data['b':'c']

b  1   -1.703054
   3   -0.809895
c  1   -0.781242
   2   -0.460427
dtype: float64

In [8]:
data.loc[['b', 'd']]
# 选取多列时传入索引列表

b  1   -1.703054
   3   -0.809895
d  2   -1.344288
   3   -0.096359
dtype: float64

In [9]:
#  在内层中进行选取
data.loc[:, 2]
#  选取了下一层索引中，索引为 2 的行

a   -0.571597
c   -0.460427
d   -1.344288
dtype: float64

    层次化索引在数据重塑和基于分组的操作（如透视表生成中）扮演者重要的角色。
    例如：通过unstack方法将这段数据重新安排到一个DataFrame中。

In [10]:
data.unstack()
# 一个新的DataFrame

,1,2,3
a,-0.070910,-0.571597,1.873524
b,-1.703054,NaN,-0.809895
c,-0.781242,-0.460427,NaN
d,NaN,-1.344288,-0.096359


    unstack的逆运算是stack

In [11]:
data.unstack().stack()

a  1   -0.070910
   2   -0.571597
   3    1.873524
b  1   -1.703054
   3   -0.809895
c  1   -0.781242
   2   -0.460427
d  2   -1.344288
   3   -0.096359
dtype: float64

In [13]:
#  对于一个DataFrame来说，每条轴都可以有分层索引：
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'], 
                             ['Green', 'Red', 'Green']])

In [14]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

    为各层定义名字

In [19]:
frame.index.names = ['key1', 'key2']
# index个数要匹配，DataFrame中有两个index，这里也需要两个

In [21]:
frame.columns.names = ['state', 'color']
# 这里也是一样的需要数量匹配

In [22]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

    获取列分组：

In [23]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

    可以单独创建MultiIndex然后复用。上面的DataFrame中的列（带有分级名称）可以这样创建：

In [25]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], 
                        ['Green', 'Red', 'Green']],
                      names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

##### 8.8.1 重排与分级排序

    swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）

In [27]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [26]:
frame.swaplevel('key1', 'key2')
#  重排了两个索引

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

    sort_index则根据单个级别中的值对数据进行排序。交换级别时也常常会用到sort_index，这样最终结果就是按照指定顺序进行字母排序了

In [50]:
frame.index

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['key1', 'key2'])

In [49]:
frame.sort_index(level=1)
# level=1 按照[1,2]来排序

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [54]:
frame.swaplevel(0,1)
# 以 levels=[['a', 'b'], [1, 2]]  列表中的下标为参数，重排了两个索引

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [56]:
frame.swaplevel(0,1).sort_index(level=0)
# 现在的索引是  数字  、 字母，level=0，按照 数字 列的数据来排序

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

    swaplevel()函数的参数：可以是index的名字（names），也可以是levels列表的对应下标

##### 8.1.2 根据级别汇总统计

    许多对DataFrame和Series的描述和汇总统计都有一个level选项，用于指定在某条轴上求和的级别。

In [59]:
#  根据行或列上的级别来进行就和
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [62]:
frame.sum(level='color', axis=1)
# 列级别上的求和，必须设置axis=1，默认的是按照axis=0（按照行）

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

    PS:pandas的groupby功能

##### 8.1.3 使用DataFrame的列进行行索引

    将DataFrame的一个或多个列当做行索引来引用；
    将行索引变成DataFrame的列。

In [63]:
frame = pd.DataFrame({
    'a': range(7),
    'b': range(7, 0, -1),
    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    'd': [0,1,2,0,1,2,3]
})

In [64]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


    DataFrame的set_index函数会将其一个或多个列转换为行索引，并重建一个新的DataFrame

In [65]:
frame2 = frame.set_index(['c', 'd'])
#  c 列的值：one ... two ...
#  d 列的值：0 1 2 0 1 2 3
#  设置以上两列为新的索引，产生一个新的DataFrame
#  原本的DataFrame的索引是整数索引

In [66]:
frame2
# 被设置为索引的列就不见了，被移除了

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [68]:
#  也可以将被设置为索引的列保留下来：
frame.set_index(['c','d'], drop=False)
# 通过drop参数的设置来完成这个功能

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

    reset_index的功能和set_index的功能刚好相反，层次化索引的级别会被转移到列里面

In [74]:
frame.reset_index()
# frame.reset_index(0)
# 有参数和无参数是一样的
#  将原本的整数索引转移到了列里面，多了index这一列

,index,a,b,c,d
0,0,0,7,one,0
1,1,1,6,one,1
2,2,2,5,one,2
3,3,3,4,two,0
4,4,4,3,two,1
5,5,5,2,two,2
6,6,6,1,two,3


In [70]:
frame
#  原来的frame还是没有变的。

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


### 8.2 合并数据集

    pandas对象中的一些数据合并：
        pandas.merge：可根据一个或多个键将不同的DataFrame中的行进行连接。（数据库的join操作）
        pandas.concat：可以沿着一条轴将多个对象堆叠在一起。
        实例方法combine_first可以将重复数据连接在一起，用一个对象中的值填充另一个对象中的缺失值。

##### 8.2.1 数据库风格的DataFrame合并

    数据集的合并(merge)或连接(join)运算是通过一个或多个键将行链接起来的。这些运算是关系型数据库（基于SQL）的核心。pandas的merge函数是对数据应用这些算法的主切入点。

In [9]:
df1 = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a' ,'a', 'b'],
    'data1': range(7)
})

In [5]:
df2 = pd.DataFrame({
    'key': ['a', 'b', 'd'],
    'data2': range(3)
})

In [10]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [7]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


    一、多对一的合并

In [11]:
#  多对一的合并
pd.merge(df1, df2)
#  没有指定用哪个列进行连接的情况下，merge会将重叠列的列名当做key。

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [12]:
#  指定用哪个列进行连接
pd.merge(df1, df2, on='key')
#  用  on  参数来进行指定

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


    如果两个对象的列名不同，也可以进行指定

In [13]:
df3 = pd.DataFrame({
    'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': range(7)
})

In [14]:
df4 = pd.DataFrame({
    'rkey': ['a', 'b', 'd'],
    'data2': range(3)
})

In [16]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


    默认情况下，merge做的是内连接，结果中的key是交集。其他的方式还有“left”、“right”、以及“outer”
    外连接求取的是key的并集，组合了左连接和右连接的效果

In [20]:
# pd.merge(df1, df2, how='outer')
pd.merge(df3, df4, how='outer', left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


|选项|说明|
|---|---|
|inner|使用两个表都有的键|
|left|使用左表中所有的键|
|right|使用右表中所有的键|
|outer|使用两个表中所有的键|

    二、多对多的合并

In [21]:
df1 = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'b'],
    'data1': range(6)
})

In [22]:
df2 = pd.DataFrame({
    'key': ['a', 'b', 'a', 'b', 'd'],
    'data2':range(5)
})

In [23]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [24]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [25]:
pd.merge(df1, df2, how='left', on='key')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


    多对多连接产生的是行的笛卡尔积。（左表3个b，右表2个b，最终3 * 2个b）

In [26]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


    要根据多个键进行合并，传入一个由列名组成的列表即可：

In [2]:
left = pd.DataFrame({
    'key1':['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': [1,2,3]
})

In [3]:
right = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one' ,'one' ,'one' ,'two'],
    'rval': [4,5,6,7]
})

In [30]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [31]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [29]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


    在进行列---列连接时：DataFrame对象中的索引会被丢弃

    重复列名的处理：可以采用手工的方式来处理（重命名轴标签），但merge函数有一个更为实用的suffixes选项，用于指定附加到左右两个DataFrame对象的重叠列名上的字符串

In [33]:
pd.merge(left, right, on='key1')
#  有重复的列名： key2  
#  自动设置为了：key2_x  key2_y

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [4]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))
# 指定 '_left', '_right'  附加到重叠列名上

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


    indicator添加特殊的列_merge，它可以指明每个行的来源，它的值有left_only, right_only,或both，根据每行的合并数据的来源

In [5]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'), indicator=True)
#  设置了indicator参数，结果显示中增加了列  _merge 

,key1,key2_left,lval,key2_right,rval,_merge
0,foo,one,1,one,4,both
1,foo,one,1,one,5,both
2,foo,two,2,one,4,both
3,foo,two,2,one,5,both
4,bar,one,3,one,6,both
5,bar,one,3,two,7,both


##### 8.2.2 索引上的合并

    DataFrame中的连接键位于索引中，此时可以传入left_index=True或right_index=True（或两个都传）以说明索引应该被用作连接键：

In [2]:
left1 = pd.DataFrame({
    'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'value': range(6)
})

In [3]:
right1 = pd.DataFrame({
    'group_val': [3.5, 7] 
}, index=['a', 'b'])

In [4]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [5]:
right1

,group_val
a,3.5
b,7.0


In [6]:
pd.merge(left1, right1, left_on='key', right_index=True)
#  right1 的连接键位于索引中，所以传入了 right_index 参数
#  默认的是取交集

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [7]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')
#  设置采取外连接，how='outer'

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


    层次化索引-----索引上的合并

In [8]:
lefth = pd.DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)
})

In [9]:
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                     index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                           [2001, 2000, 2000, 2000, 2001, 2002]],
                     columns=['event1', 'event2'])

In [10]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [11]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

    层次化索引，必须以列表的形式指明用作合并键的多个列（注意用how='outer'对重复索引值的处理）

In [12]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)
#  以左表的 key1  key2  列作为合并列
#  merge默认的交集
# righth有层次化索引，连接键位于其索引之中，传入了right_index=True

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [13]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')
# 设置为外连接

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [16]:
#  同时使用合并双方的索引
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                    columns=['Ohio', 'Nevada'])

In [17]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [18]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14]],
                     index=['b', 'c', 'd', 'e'],
                     columns=['Missouri', 'Alabama'])

In [19]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [20]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)
#  要合并的列是索引列，因此left和right都要设置index

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


    DataFrame的join方法，更方便的实现按索引合并。
    还可以用于合并多个带有相同或相似索引的DataFrame对象，但要求没有重叠的列。
    还支持在调用的DataFrame列上，连接传递的DataFrame索引。

In [24]:
# left2.join(right2)
# join方法默认使用左连接， 保留左边表的行索引
left2.join(right2, how='outer')
# 设置为外连接

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [25]:
left1.join(right1, on='key')
#  连接列：左表中是key列，右表中是索引列，
# join方法是按照索引列来进行连接的，现在左表的连接列不是索引，所以要设置一下'on'

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [26]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [27]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [29]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [28]:
left2.join([right2, another])
#  以列表的形式传入两个DataFrame，实现三个DataFrame的连接（默认内连接）

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [34]:
# left2.join([right2, another], how='outer', sort=False)
# 第一次没有传入sort=False的话，会有警告
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


##### 8.2.3 轴向连接
    另一种数据合并也被称作连接、绑定或堆叠。

In [3]:
arr = np.arange(12).reshape((3,4))

In [4]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [5]:
np.concatenate([arr, arr], axis=1)
#  使用concatenate函数连接了两个数组

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

    对于pandas对象，带有标签的轴使你能够进一步推广数组的连接运算。
    
    首先看----Series----

In [2]:
s1 = pd.Series([0, 1], index=['a', 'b'])

In [3]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

In [4]:
s3 = pd.Series([5, 6], index=['f', 'g'])

In [5]:
#  调用cancat函数将三个Series连接
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

    默认情况下concat是在axis=0上工作的，最终会产生一个新的Series。传入axis=1，结果就变成一个DataFrame

In [7]:
pd.concat([s1, s2, s3], axis=1, sort=False)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [8]:
s4 = pd.concat([s1, s3])

In [9]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [11]:
pd.concat([s1, s4], axis=1, sort=False)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [12]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [14]:
#  join_axes指定要在其他轴上使用的索引
pd.concat([s1, s4], axis=1, join_axes=[['a', 'b', 'c', 'e']])

,0,1
a,0.0,0.0
b,1.0,1.0
c,NaN,NaN
e,NaN,NaN


    创建层次化索引----keys参数的设置

In [16]:
#  在连接轴上创建层次化索引，使用keys参数：
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result
#  默认的axis=0，列上面的操作

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [20]:
type(result)

pandas.core.series.Series

In [17]:
result.unstack()
#  unstack()函数： 理解为：将数据从花括号结构转变为表格结构（Series ----> DataFrame）

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


    沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头：

In [24]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'], sort=False)

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


    再看-----DataFrame----

In [2]:
df1 = pd.DataFrame(np.arange(6).reshape((3,2)), index=['a','b','c'],
                  columns=['one','two'])
df1
#  这个reshape函数,试了一下，参数直接写  3,2  或者写(3,2)，效果都一样

,one,two
a,0,1
b,2,3
c,4,5


In [3]:
df2 = pd.DataFrame(5+np.arange(4).reshape(2,2), index=['a', 'c'],
                  columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [4]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort=False)
# keys参数，创建了层次化索引，在行上面创建

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

    如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值

In [6]:
pd.concat({'level1': df1, 'level2': df2}, axis=1, sort=False)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

    names参数命名创建的轴级别：

In [8]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'], sort=False)
# namse参数设置了两层索引的名字

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

    DataFrame的行索引不包含任何相关数据：
    传入：ignore_index=True

In [11]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,-0.221112,-0.118709,0.552203,-0.557141
1,0.867813,0.066338,-0.035867,1.602709
2,-0.813195,-0.426063,-0.443583,1.058433


In [12]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,-1.358626,0.131928,1.567449
1,-1.652450,0.022204,1.203019


In [14]:
pd.concat([df1, df2], ignore_index=True, sort=False)

,a,b,c,d
0,-0.221112,-0.118709,0.552203,-0.557141
1,0.867813,0.066338,-0.035867,1.602709
2,-0.813195,-0.426063,-0.443583,1.058433
3,1.567449,-1.358626,NaN,0.131928
4,1.203019,-1.652450,NaN,0.022204


##### 8.2.4 合并重叠数据

    where函数，表示一种等价于面向数组的if-else：

In [16]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
             index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [17]:
b = pd.Series(np.arange(len(a), dtype=np.float64), 
             index=['f', 'e', 'd', 'c', 'b', 'a'])
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [18]:
b[-1] = np.nan
#  设置最后一个数据为nan

In [19]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [20]:
np.where(pd.isnull(a), b, a)
#  如果a为空，则选取b的值，否则选取a的值

array([0. , 2.5, 2. , 3.5, 4.5, nan])

    Series有一个combine_first方法，实现的也是一样的功能，还带有pandas的数据对齐功能

In [21]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

    对于DataFrame：用传递对象中的数据为调用对象的缺失数据“打补丁”

In [22]:
df1 = pd.DataFrame({
    'a': [1., np.nan, 5., np.nan],
    'b': [np.nan, 2., np.nan, 6.],
    'c': range(2, 18, 4)
})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [23]:
df2 = pd.DataFrame({
    'a': [5., 4, np.nan, 3., 7.],
    'b': [np.nan, 3., 4., 6., 8.]
})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [24]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 8.3 重塑和轴向旋转

    重新排列表格型数据的基础运算。这些函数也称作重塑（reshape）或轴向旋转（pivot）运算

##### 8.3.1 重塑层次化索引

    层次化索引主要功能：
    stack：将数据的“列”旋转为行
    unstack：将数据的“行”旋转为列

In [26]:
data = pd.DataFrame(np.arange(6).reshape((2,3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                   columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


    运用stack方法即可将列转换为行：得到的是一个Series。

In [27]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

    对于Series，可以用unstack()将其重排为一个DataFrame

In [ ]:
reslut

##### 8.3.2 将“长格式”旋转为“宽格式”

##### 8.3.3 将“宽格式”旋转为“长格式”